In [1]:
import sys, re, math, time
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import collections
from collections import OrderedDict
from matplotlib.pyplot import cm
%load_ext autoreload
%autoreload 2
from run_experiments import *
from datahelper import *
import keras.metrics
keras.metrics.cindex_score = cindex_score
from keras.models import load_model 
fpath = '../data/davis/'
%matplotlib inline

Using TensorFlow backend.


In [2]:
ligands = json.load(open(fpath+"ligands_can.txt"), object_pairs_hook=OrderedDict)
proteins = json.load(open(fpath+"proteins.txt"), object_pairs_hook=OrderedDict)

Y = pickle.load(open(fpath + "Y","rb"), encoding='latin1') ### TODO: read from raw

In [3]:
perfmeasure = get_cindex
runmethod = build_combined_categorical
FLAGS = argparser()
FLAGS.log_dir = FLAGS.log_dir + str(time.time()) + "/"
FLAGS.num_windows = 32
FLAGS.seq_window_lengths = [4]
FLAGS.smi_window_lengths = [4]
FLAGS.batch_size = 256
FLAGS.num_epoch = 100
FLAGS.max_seq_len = 1000
FLAGS.max_smi_len = 100
FLAGS.dataset_path= '../data/davis/'
FLAGS.problem_type = 1
FLAGS.log_dir = '../logs'


In [23]:
dataset = DataSet( fpath = FLAGS.dataset_path, ### BUNU ARGS DA GUNCELLE
                      setting_no = FLAGS.problem_type, ##BUNU ARGS A EKLE
                      seqlen = FLAGS.max_seq_len,
                      smilen = FLAGS.max_smi_len,
                      need_shuffle = False )
# set character set size
FLAGS.charseqset_size = dataset.charseqset_size 
FLAGS.charsmiset_size = dataset.charsmiset_size 

XD, XT, Y = dataset.parse_data(fpath = FLAGS.dataset_path)

XD = np.asarray(XD)
XT = np.asarray(XT)
Y = np.asarray(Y)

drugcount = XD.shape[0]
print(drugcount)
targetcount = XT.shape[0]
print(targetcount)

FLAGS.drug_count = drugcount
FLAGS.target_count = targetcount

label_row_inds, label_col_inds = np.where(np.isnan(Y)==False)  #basically finds the point address of affinity [x,y]

if not os.path.exists(figdir):
    os.makedirs(figdir)
test_set, outer_train_sets = dataset.read_sets(FLAGS.dataset_path, FLAGS.problem_type) 


Reading ../data/davis/ start
Read ../data/davis/ start
68
442
Reading ../data/davis/ start


In [24]:

logging("---Parameter Search-----", FLAGS)

Y = np.mat(np.copy(Y))

params = {}

train_drugs, train_prots,  train_Y = prepare_interaction_pairs(XD, XT, Y, label_row_inds, label_col_inds)

In [25]:

data_path='../../data/'
XD_dtc, XT_dtc, Y_dtc = get_DTC_train(data_path+'dtc_for_deepDTA.csv', FLAGS.max_smi_len, FLAGS.max_seq_len)


In [26]:
from sklearn.model_selection import train_test_split
all_train_drugs = np.concatenate((np.asarray(train_drugs), np.asarray(XD_dtc)), axis=0)
all_train_prots = np.concatenate((np.asarray(train_prots), np.asarray(XT_dtc)), axis=0)
all_train_Y = np.concatenate((np.asarray(train_Y), np.asarray(Y_dtc)), axis=0)
all_train_Y = -np.log10(all_train_Y/1e9)

XD_train, XD_val, XT_train, XT_val, Y_train, Y_val = train_test_split(all_train_drugs, all_train_prots, all_train_Y, 
                                                                      test_size=0.1, 
                                                                     random_state=15)


In [8]:
model_name='checkpoints/davis_dtc_dta'
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)
checkpoint_callback = ModelCheckpoint(model_name+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

gridmodel = build_combined_categorical(FLAGS, FLAGS.num_windows, FLAGS.smi_window_lengths[0], FLAGS.seq_window_lengths[0])
gridres = gridmodel.fit(([XD_train, XT_train ]), Y_train, batch_size=FLAGS.batch_size, epochs=1000, 
        validation_data=( ([np.array(XD_val), np.array(XT_val) ]), np.array(Y_val))
           , callbacks=[early_stopping_callback, checkpoint_callback], verbose=2)

gridmodel.save('davis_dtc_dta.h5')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Train on 41310 samples, validate on 4591 samples
Epoch 1/1000
Epoch 00001: val_loss improved from inf to 0.88949, saving model to checkpoints/davis_dtc_dta.h5
 - 36s - loss: 2.0722 - cindex_score: 0.5301 - val_loss: 0.8895 - val_cindex_score: 0.4842
Epoch 2/1000
Epoch 00002: val_loss did not improve
 - 30s - loss: 1.0628 - cindex_score: 0.5643 - val_loss: 1.0610 - val_cindex_score: 0.5156
Epoch 3/1000
Epoch 00003: val_loss did not improve
 - 30s - loss: 0.9197 - cindex_score: 0.5952 - val_loss: 1.5839 - val_cindex_score: 0.5760
Epoch 4/1000
Epoch 00004: val_loss improved from 0.88949 to 0.85833, saving model to checkpoints/davis_dtc_dta.h5
 - 30s - loss: 0.8775 - cindex_score: 0.6119 - val_loss: 0.8583 - val_cindex_score: 0.6624
Epoch 5/1000
Epoch 00005: val_loss did not improve
 - 30s - loss: 0.8313 - cindex_score: 0.6268 - val_loss: 0.8945 

Epoch 00047: val_loss did not improve
 - 30s - loss: 0.5611 - cindex_score: 0.7395 - val_loss: 0.6083 - val_cindex_score: 0.7237
Epoch 48/1000
Epoch 00048: val_loss did not improve
 - 29s - loss: 0.5634 - cindex_score: 0.7418 - val_loss: 0.5845 - val_cindex_score: 0.7374
Epoch 49/1000
Epoch 00049: val_loss improved from 0.58159 to 0.57938, saving model to checkpoints/davis_dtc_dta.h5
 - 30s - loss: 0.5574 - cindex_score: 0.7427 - val_loss: 0.5794 - val_cindex_score: 0.7325
Epoch 50/1000
Epoch 00050: val_loss did not improve
 - 29s - loss: 0.5540 - cindex_score: 0.7429 - val_loss: 0.5828 - val_cindex_score: 0.7264
Epoch 51/1000
Epoch 00051: val_loss did not improve
 - 29s - loss: 0.5573 - cindex_score: 0.7422 - val_loss: 0.6048 - val_cindex_score: 0.7287
Epoch 52/1000
Epoch 00052: val_loss did not improve
 - 30s - loss: 0.5586 - cindex_score: 0.7405 - val_loss: 0.6072 - val_cindex_score: 0.7296
Epoch 53/1000
Epoch 00053: val_loss did not improve
 - 29s - loss: 0.5546 - cindex_score: 0.7

In [9]:
from sklearn.metrics import mean_squared_error, auc, f1_score
from scipy.stats import pearsonr, spearmanr

gridmodel = load_model('checkpoints/davis_dtc_dta.h5')
predicted_labels = gridmodel.predict([np.array(XD_val), np.array(XT_val) ])
loss, rperf2 = gridmodel.evaluate(([np.array(XD_val),np.array(XT_val) ]), np.array(Y_val), verbose=0)


In [10]:
predicted_labels = gridmodel.predict([np.array(XD_val), np.array(XT_val) ])
print('cindex:', get_cindex(Y_val, predicted_labels))
print('rmse:', np.sqrt(mean_squared_error(Y_val, predicted_labels)))
#print('pearsonr:', pearsonr(Y_val, predicted_labels[:, 0]))
#print('spearmanr:', np.sqrt(spearmanr(val_Y, predicted_labels[:, 0])))
print('f1:', np.sqrt(f1_score(Y_val>7, predicted_labels>7)))


cindex: [0.72608912]
rmse: 0.7713228556107431
f1: 0.6403685988913285


In [30]:
XD_test, XT_test = get_DTC_train('../data/dtc_test_for_deepDTA.csv', FLAGS.max_smi_len, FLAGS.max_seq_len, with_label=False)
XD_test, XT_test = np.asarray(XD_test), np.asarray(XT_test)
predicted_labels = gridmodel.predict([np.array(XD_test), np.array(XT_test) ])

In [31]:
submission_data = pd.read_csv(data_path+'round_1_template.csv')
submission_data.loc[:, 'pKd_[M]_pred'] = predicted_labels
submission_data.to_csv('../data/submission_file3.csv', index=None)

In [32]:
np.sum(predicted_labels>7)

39